# **Precio promedio - Recámaras**

In [101]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "alfa" in nombre and "queretaro" in nombre and "2024" in nombre

    #if nombre.startswith("df_") and "pachuca" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_queretaro',
 'df_alfa_agosto_2024_queretaro',
 'df_alfa_marzo_2024_queretaro',
 'df_alfa_mayo_2024_queretaro']

In [102]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Preciol_23':'precio','Baño_total': 'baño_total','Banos_Total':'baño_total','Precio':'precio','recamaras':'Recamaras'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [103]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    #    segment_df = globals()[nombre_df][['precio', 'Recamaras','Tipo']]
    segment_df = globals()[nombre_df][['id', 'Categoria', 'precio', 'propiedad', 'metros_total',
       'metros_construido', 'precio_m2_terreno', 'precio_m2_construido',
       'tiempo_de_publicacion', 'Estado', 'Tipo', 'Estacionamientos',
       'Recamaras', 'Banos', 'Medio_banos', 'baño_total', 'Seguridad',
       'Colonia', 'CP', 'url']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)
# Concatenar todos los DataFrames individuales en uno solo
df_concatenado = pd.concat(dataframes_list)

## Eliminación de duplicados

In [104]:
# Eliminar por duplicado general, osea los registros exactamente iguales
print(df_concatenado[df_concatenado.duplicated()])

Empty DataFrame
Columns: [id, Categoria, precio, propiedad, metros_total, metros_construido, precio_m2_terreno, precio_m2_construido, tiempo_de_publicacion, Estado, Tipo, Estacionamientos, Recamaras, Banos, Medio_banos, baño_total, Seguridad, Colonia, CP, url]
Index: []


In [105]:
# Duplicado por variables, osea los registros exactamente iguales con los valores de variables
print(df_concatenado[df_concatenado.duplicated(subset=['metros_construido','Colonia', 'CP', 'precio','Estacionamientos','Recamaras','baño_total','propiedad'])])

        id Categoria    precio  \
49      50        E2    730000   
50      51        E2    730000   
90      91              750000   
91      92              750000   
110    111              750000   
...    ...       ...       ...   
9423  9424        B2   2950000   
9428  9429        S1   5900000   
9435  9436        S3  11250000   
9441  9442        L+  21000000   
9547  9548        E2    587409   

                                                                                                       propiedad  \
49                                                                         venta de casa en juriquilla queretaro   
50                                                    venta de casa en col cumbres del lago juriquilla queretaro   
90                                          excelente casa en venta lago yalahan cumbres del lago juriquilla qro   
91                                                                 venta de hermosa casa en juriquilla queretaro   
110   casa 

In [106]:
pd.set_option('display.max_colwidth', None) # Mostrar completas
print(df_concatenado[df_concatenado.duplicated(subset=['metros_construido','Colonia', 'CP', 'precio','Estacionamientos','Recamaras','baño_total'])]['propiedad'].unique())

['casa esquinera 15min del centro de queretaro'
 'venta de casa en queretaro juquilla col cumbres del lago'
 'linda casa en santiago de queretaro qro' ...
 'hermosa y amplia casa con hermosas vistas'
 'en venta hermosa propiedad en queretaro'
 'hermosa casa en venta en queretaro']


In [107]:
# Agrupar por los registros duplicados y contar las ocurrencias
agrupados = df_concatenado[df_concatenado.duplicated(subset=['metros_construido','Colonia', 'CP', 'precio','Estacionamientos','Recamaras','propiedad'], keep=False)].groupby(['metros_construido','Colonia', 'CP', 'precio','Recamaras','propiedad']).size().reset_index(name='count')
print(agrupados)

      metros_construido                           Colonia CP    precio  \
0                     0                        campanario  0  27500000   
1                     0                  cumbres del lago  0   2250000   
2                     0                        el refugio      4736059   
3                     0                   jurica misiones  0    830000   
4                     0                         queretaro      1542000   
...                 ...                               ... ..       ...   
3598               1699  fraccionamiento villas del meson     49000000   
3599               1699                  villas del meson  0  49000000   
3600               1870                     el campanario     59900000   
3601               1932      fracc balcones de juriquilla     17000000   
3602               3112                  cumbres del lago      5450000   

      Recamaras  \
0             3   
1             3   
2             3   
3             2   
4             2 

In [108]:
# Eliminar duplicados, conservando la primera aparición
df_sin_duplicados = df_concatenado.drop_duplicates(subset=['metros_construido','Colonia', 'CP', 'precio','Estacionamientos','Recamaras','baño_total','propiedad'], keep='first')
# conservar la última aparición
# df_sin_duplicados = df_concatenado.drop_duplicates(keep='last')
print(df_sin_duplicados)

        id Categoria   precio                                   propiedad  \
0        1        B1  2640000    casa venta biogrand juriquilla queretaro   
1        2        S2  6950000       venta de casa en la colonia cimatario   
2        3        B2  2908782              casa  villa villas del refugio   
3        4        E1   280000            venta de departamento tim 708520   
4        5        E1   400000                 casa en venta de una planta   
...    ...       ...      ...                                         ...   
9624  9625        E3   953000    vomo emb casa en venta en col modelo qro   
9625  9626        C1  1950000  loma bonita casa venta queretaro queretaro   
9626  9627        A3  3772000   del pinar santiago de queretaro queretaro   
9627  9628        C1  1800000                    adjudicacion hipotecaria   
9628  9629        E2   730000          hermosa casa en venta en queretaro   

      metros_total  metros_construido  precio_m2_terreno  \
0              

## Diidir por tipo

In [109]:
df_sin_duplicados['Tipo'].unique()

array(['Casa', 'Departamento', 'Casa En Fraccionamiento', 'Duplex',
       'Condominio Horizontal', 'Casa En Condominio', 'Casa Duplex'],
      dtype=object)

In [110]:
casas = df_sin_duplicados[df_sin_duplicados['Tipo'].isin(['Casa', 'Casa En Fraccionamiento', 'Casa En Condominio', 'Casa Duplex'])]
depas = df_sin_duplicados[df_sin_duplicados['Tipo'].isin(['Departamento', 'Condominio Horizontal','Duplex'])]

Ver que las cantidades coinciden por el tipo

In [111]:
casas_precio = casas['precio']
casas_precio.sort_values() 

1            290000
2            328000
3            336528
4            345368
5            350000
            ...    
10992      87963509
10991      87963509
8406       92500000
8407      121200000
10997    9680000000
Name: precio, Length: 24939, dtype: int64

In [112]:
depas_precio = depas['precio']
depas_precio.sort_values()

3            280000
0            280000
3            280000
0            400000
1            440450
            ...    
8409      520012341
10995    3100000000
8410     3150000000
8411     4280000000
10996    4450000000
Name: precio, Length: 9518, dtype: int64

# Resultados

## Casas

In [113]:
casas['Recamaras'] = casas['Recamaras'].astype(int)
casas = casas[(casas['Recamaras'] >= 1) & (casas['Recamaras'] <= 7)]
casas['Recamaras'].unique()

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13084\4152421137.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



array([4, 3, 2, 6, 5, 1, 7])

In [114]:
# Ajustar los valores para que no superen un #
casas.loc[:, 'Recamaras'] = casas['Recamaras'].clip(lower=1,upper=8)
# Calcular el promedio de precio por el número único 
df_promedio_casas = casas.groupby('Recamaras')['precio'].mean().reset_index()
df_promedio_casas

,Recamaras,precio
0,1,5.645664e+06
1,2,2.554781e+06
2,3,4.928178e+06
3,4,9.909266e+06
4,5,1.019563e+07
5,6,1.044534e+07
6,7,1.661260e+07


In [115]:
import pandas as pd
import plotly.graph_objects as go
import locale

# Establecer configuración regional para el formato de precios
locale.setlocale(locale.LC_ALL, '')

# Definir función para formatear precios
def format_price(price):
    return locale.currency(price, grouping=True)

# Crear una lista de etiquetas para el eje x
etiquetas = [str(recamaras) if recamaras != 16 else '16+' for recamaras in df_promedio_casas['Recamaras']]

# Obtener los precios y el número de recámaras
precios = df_promedio_casas['precio']

# Crear el texto con los precios formateados
text_precios = [format_price(precio) for precio in precios]

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff', '#ff0059']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=etiquetas,
    y=precios,
    text=text_precios,
    marker_color=colores,
    textposition='auto' 
))
fig.update_layout(
    #title='Precio promedio por número de baños',
    xaxis=dict(title='Recámaras'),
    yaxis=dict(title='$', gridcolor='#dddcda', ),
    bargap=0.1,
    bargroupgap=0.05,
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

fig.show()

In [116]:
casas = casas.drop_duplicates(subset=['precio', 'Recamaras', 'Tipo'])
casas[['precio','Recamaras']].sort_values(by='precio')

,precio,Recamaras
1,290000,2
2,328000,2
3,336528,3
4,345368,2
5,350000,2
...,...,...
10989,80000000,4
10991,87963509,3
8406,92500000,3
8407,121200000,3


In [117]:
casas = pd.concat([casas[casas['Recamaras'] != 7], casas[casas['Recamaras'] == 7].drop_duplicates(subset=['precio', 'Recamaras', 'Tipo'])])
recam_five = casas[casas['Recamaras']==7]
recam_five[['precio','Recamaras']].sort_values(by='precio')

,precio,Recamaras
1121,2050000,7
1219,2200000,7
2881,3650000,7
4888,4200000,7
3799,4200000,7
7101,5057000,7
4000,5750000,7
8907,7500000,7
9507,8000000,7
7844,8900000,7


In [118]:
registro_especifico = casas[casas['precio']==38000000]
registro_especifico[['precio','Recamaras']]

,precio,Recamaras
8403,38000000,4
7690,38000000,4
10938,38000000,7


In [119]:
casas = casas[~((casas['precio']==32000000) & (casas['Recamaras']==1))]
casas = casas[~((casas['precio']==8100000) & (casas['Recamaras']==1))]
casas = casas[~((casas['precio']==7750000) & (casas['Recamaras']==1))]
casas = casas[~((casas['precio']==7000000) & (casas['Recamaras']==1))]
casas = casas[~((casas['precio']==5974650) & (casas['Recamaras']==1))]
casas = casas[~((casas['precio']==5818080) & (casas['Recamaras']==1))]
casas = casas[~((casas['precio']==5200000) & (casas['Recamaras']==1))]
casas = casas[~((casas['precio']==4600000) & (casas['Recamaras']==1))]
casas = casas[~((casas['precio']==9680000000) & (casas['Recamaras']==4))]
casas = casas[~((casas['precio']==60000000) & (casas['Recamaras']==7))]
casas = casas[~((casas['precio']==38000000) & (casas['Recamaras']==7))]
casas = casas[~((casas['precio']==40000000) & (casas['Recamaras']==7))]

recam_five = casas[casas['Recamaras']==1].sort_values(by='precio')
recam_five[['precio', 'Recamaras', 'Tipo']]

,precio,Recamaras,Tipo
31,418000,1,Casa
247,610000,1,Casa
987,980000,1,Casa
400,990000,1,Casa
1030,1005000,1,Casa
803,1630000,1,Casa
1044,1950000,1,Casa
2356,2350000,1,Casa
3711,2530000,1,Casa
3159,2600000,1,Casa


In [120]:
# Ajustar los valores para que no superen un #
casas.loc[:, 'Recamaras'] = casas['Recamaras'].clip(lower=1,upper=8)
# Calcular el promedio de precio por el número único 
df_promedio_casas = casas.groupby('Recamaras')['precio'].mean().reset_index()
df_promedio_casas

,Recamaras,precio
0,1,2.199214e+06
1,2,2.382302e+06
2,3,4.561825e+06
3,4,8.830944e+06
4,5,9.946304e+06
5,6,1.163107e+07
6,7,1.492491e+07


In [121]:
import pandas as pd
import plotly.graph_objects as go
import locale

# Establecer configuración regional para el formato de precios
locale.setlocale(locale.LC_ALL, '')

# Definir función para formatear precios
def format_price(price):
    return locale.currency(price, grouping=True)

# Crear una lista de etiquetas para el eje x
etiquetas = [str(recamaras) if recamaras != 16 else '16+' for recamaras in df_promedio_casas['Recamaras']]

# Obtener los precios y el número de recámaras
precios = df_promedio_casas['precio']

# Crear el texto con los precios formateados
text_precios = [format_price(precio) for precio in precios]

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff', '#ff0059']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=etiquetas,
    y=precios,
    text=text_precios,
    marker_color=colores,
    textposition='auto' 
))
fig.update_layout(
    #title='Precio promedio por número de baños',
    xaxis=dict(title='Recámaras'),
    yaxis=dict(title='$', gridcolor='#dddcda', ),
    bargap=0.1,
    bargroupgap=0.05,
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

fig.show()

In [122]:
# Función para guardar la gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    import os
    import plotly.io as pio

    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Ejemplo de uso
guardar_grafico_como_html(fig, 'g_bar_precio_promedio_recamaras_casa',  carpeta='assets/graficas')


## Depa

In [123]:
depas['Recamaras'] = depas['Recamaras'].astype(int)
depas = depas[(depas['Recamaras'] >= 1) & (depas['Recamaras'] <= 7)]
depas['Recamaras'].unique()

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13084\2124676156.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



array([2, 3, 1, 5, 4, 6])

In [124]:
# Ajustar los valores para que no superen un #
depas.loc[:, 'Recamaras'] = depas['Recamaras'].clip(lower=1,upper=8)
# Calcular el promedio de precio por el número único 
df_promedio_depas = depas.groupby('Recamaras')['precio'].mean().reset_index()
df_promedio_depas

,Recamaras,precio
0,1,3.051919e+06
1,2,5.095610e+06
2,3,6.096612e+06
3,4,2.040680e+07
4,5,7.776643e+06
5,6,2.514000e+07


In [125]:
import pandas as pd
import plotly.graph_objects as go
import locale

# Establecer configuración regional para el formato de precios
locale.setlocale(locale.LC_ALL, '')

# Definir función para formatear precios
def format_price(price):
    return locale.currency(price, grouping=True)

# Crear una lista de etiquetas para el eje x
etiquetas = [str(recamaras) if recamaras != 16 else '16+' for recamaras in df_promedio_depas['Recamaras']]

# Obtener los precios y el número de recámaras
precios = df_promedio_depas['precio']

# Crear el texto con los precios formateados
text_precios = [format_price(precio) for precio in precios]

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff', '#ff0059']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=etiquetas,
    y=precios,
    text=text_precios,
    marker_color=colores,
    textposition='auto' 
))
fig.update_layout(
    #title='Precio promedio por número de baños',
    xaxis=dict(title='Recámaras'),
    yaxis=dict(title='$', gridcolor='#dddcda', ),
    bargap=0.1,
    bargroupgap=0.05,
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

fig.show()

In [126]:
depas = pd.concat([depas[depas['Recamaras'] != 4], depas[depas['Recamaras'] == 4].drop_duplicates(subset=['precio', 'Recamaras', 'Tipo'])])
depas[['precio','Recamaras']].sort_values(by='precio')

,precio,Recamaras
3,280000,2
0,280000,2
3,280000,2
0,400000,2
1,440450,2
...,...,...
8409,520012341,2
10995,3100000000,4
8410,3150000000,2
8411,4280000000,3


In [129]:
recam_five = depas[depas['Recamaras']==4].sort_values(by='precio')
recam_five[['precio', 'Recamaras', 'Tipo']]

,precio,Recamaras,Tipo
2737,2650000,4,Departamento
2811,2700000,4,Departamento
3133,2851000,4,Departamento
3608,3100000,4,Departamento
3651,3103955,4,Departamento
...,...,...,...
10770,23000000,4,Departamento
8320,26600000,4,Condominio Horizontal
8369,32950000,4,Condominio Horizontal
8398,58500000,4,Departamento


In [134]:
registro_especifico = depas[depas['precio']==58500000]
registro_especifico[['precio','Recamaras']]

,precio,Recamaras
8398,58500000,4


In [135]:
depas = depas[~((depas['precio']==3100000000) & (depas['Recamaras']==4))]
depas = depas[~((depas['precio']==58500000) & (depas['Recamaras']==4))]

In [140]:
depas = pd.concat([depas[depas['Recamaras'] != 6], depas[depas['Recamaras'] == 6].drop_duplicates(subset=['precio', 'Recamaras', 'Tipo'])])
recam_five = depas[depas['Recamaras']==6].sort_values(by='precio')
recam_five[['precio', 'Recamaras', 'Tipo']]

,precio,Recamaras,Tipo
3984,5700000,6,Departamento
7842,30000000,6,Departamento


In [142]:
# Ajustar los valores para que no superen un #
depas.loc[:, 'Recamaras'] = depas['Recamaras'].clip(lower=1,upper=8)
# Calcular el promedio de precio por el número único 
df_promedio_depas = depas.groupby('Recamaras')['precio'].mean().reset_index()
df_promedio_depas

,Recamaras,precio
0,1,3.051919e+06
1,2,5.095610e+06
2,3,6.096612e+06
3,4,7.533238e+06
4,5,7.776643e+06
5,6,1.785000e+07


In [144]:
import pandas as pd
import plotly.graph_objects as go
import locale

# Establecer configuración regional para el formato de precios
locale.setlocale(locale.LC_ALL, '')

# Definir función para formatear precios
def format_price(price):
    return locale.currency(price, grouping=True)

# Crear una lista de etiquetas para el eje x
etiquetas = [str(recamaras) if recamaras != 16 else '16+' for recamaras in df_promedio_depas['Recamaras']]

# Obtener los precios y el número de recámaras
precios = df_promedio_depas['precio']

# Crear el texto con los precios formateados
text_precios = [format_price(precio) for precio in precios]

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff', '#ff0059']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=etiquetas,
    y=precios,
    text=text_precios,
    marker_color=colores,
    textposition='auto' 
))
fig.update_layout(
    #title='Precio promedio por número de baños',
    xaxis=dict(title='Recámaras'),
    yaxis=dict(title='$', gridcolor='#dddcda', ),
    bargap=0.1,
    bargroupgap=0.05,
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

fig.show()

In [145]:
# Función para guardar la gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    import os
    import plotly.io as pio

    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Ejemplo de uso
guardar_grafico_como_html(fig, 'g_bar_precio_promedio_recamaras_depa',  carpeta='assets/graficas')
